In [1]:
#Import the libraries
# import math
import pandas_datareader as pdr
import yfinance as yf
import datetime
# import mplfinance as mpf

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

In [2]:
# df['MA5'] = df.Close.rolling(5).mean()
# df['MA20'] = df.Close.rolling(20).mean()

# Pivot = (High + Low + Close)/3
# Bottom CPR = (High + Low)/ 2
# Top CPR = (Pivot – BC) + Pivot

#To calculate the Base Pivot Point: Pivot Point (P) = (High + Low + Close)/3 
#To calculate the First Support Level: Support 1 (S1) = P – {.382 * (High  –  Low)} 
#To calculate the Second Support Level: Support 2 (S2) = P – {.618 * (High  –  Low)} 
#To calculate the First Resistance Level: Resistance 1 (R1) = P + {.382 * (High  –  Low)} 
#To calculate the Second Resistance Level: Resistance 2 (R2) = P + {.618 * (High  –  Low)} 
#To calculate the Third Resistance Level: Resistance 3 (R3) = P + {1 * (High  –  Low)}

In [3]:
# Top 500 companies
df = pd.read_excel("Nifty 500.xlsx")
df = df[df['Volume']>500000]
print(df.shape[0])

288


In [4]:
# Top 200 companies
df = pd.read_csv("ind_nifty200list.csv")
df.columns.values[2] = "Ticker"

In [5]:
# # Top 100 companies
# df = pd.read_csv("ind_nifty100list.csv")
# df.columns.values[2] = "Ticker"

In [6]:
values = ["INDUSTOWER",]
df.drop(df[df['Ticker'].isin(values)].index, inplace = True)

In [7]:
now = datetime.datetime.now()
last_month = now.month-1 if now.month > 1 else 12
last_month_start = str(now.year) + '-' + str(last_month) + '-01'
last_month_end = (datetime.date(now.year, now.month, 1) - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

current_month_start = str(now.year) + '-' + str(now.month) + '-01'
current_month_end = datetime.datetime.now().strftime("%Y-%m-%d")

In [8]:
def stock_data(ticker, start, end):
    stock = pdr.DataReader(ticker+".NS", data_source='yahoo', start=start, end=end)
#     stock = yf.Ticker(ticker+".NS").history(start=start, end=end)
    return stock

In [9]:
def pivot_points(stock):
    
    high = stock['High'].max()
    low = stock['Low'].min()
    close = stock['Close'].iloc[-1]
    pivot = (high + low + close)/3
    pivotBottom = (high + low)/2
    pivotTop = (pivot - pivotBottom) + pivot
    s1 = pivot - (.382 * (high  -  low))
    s2 = pivot - (.618 * (high  -  low))
    s3 = pivot - (1 * (high  -  low))
    r1 = pivot + (.382 * (high  -  low))
    r2 = pivot + (.618 * (high  -  low))
    r3 = pivot + (1 * (high  -  low))
    
    pivotDict = {
        'Pivot': pivot, 'Pivot Bottom': pivotBottom, 'Pivot Top': pivotTop, 
        'S1': s1, 'S2': s2, 'S3': s3, 'R1': r1, 'R2': r2, 'R3': r3
    }
    
    return pivotDict

In [10]:
def stocks_position(ticker):
    
    stock_last_month = stock_data(ticker, last_month_start, last_month_end) # Getting last month stock data
    pivotPoints = pivot_points(stock_last_month) # Calculating Pivot points from last month data

    stock_current_month = stock_data(ticker, current_month_start, current_month_end) # Getting current month stock data
    if stock_current_month['Close'].iloc[-1] < pivotPoints['Pivot']:
        if stock_current_month['Close'].iloc[-1] < pivotPoints['S1']:
            if stock_current_month['Close'].iloc[-1] < pivotPoints['S2']:
                if stock_current_month['Close'].iloc[-1] < pivotPoints['S3']:
                    return ticker + ' Below S3'
                return ticker + ' Below S2'
            return ticker + ' Below S1'
#         return ticker + ' Below Pivot'

In [11]:
for i in df['Ticker']:
    position = stocks_position(i)
    if position != None:
        print(position)

ABCAPITAL Below S1
AMARAJABAT Below S1
AMBUJACEM Below S1
APOLLOHOSP Below S1
ASIANPAINT Below S1
BIOCON Below S1
BBTC Below S1
BRITANNIA Below S3
CADILAHC Below S2
CIPLA Below S2
COLPAL Below S2
CONCOR Below S1
CROMPTON Below S1
CUMMINSIND Below S2
DABUR Below S2
DALBHARAT Below S1
EICHERMOT Below S1
EMAMILTD Below S2
GODREJCP Below S1
HDFCLIFE Below S1
HINDUNILVR Below S3
ICICIGI Below S2
ICICIPRULI Below S2
IGL Below S1
MGL Below S1
MFSL Below S1
NAVINFLUOR Below S1
NESTLEIND Below S1
PETRONET Below S1
PFIZER Below S2
SHREECEM Below S1
TCS Below S2
TORNTPHARM Below S1
ULTRACEMCO Below S1


In [2]:
yf.Ticker("SAIL.NS").history(threads = False,start='2021-07-12', end='2021-07-13')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-07-12,126.349998,127.400002,124.349998,124.849998,28310351,0,0


In [3]:
pdr.DataReader("SAIL.NS", data_source='yahoo', start='2021-07-12', end='2021-07-13')

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-07-12,127.400002,124.349998,126.349998,124.849998,28310351,124.849998
2021-07-13,127.250000,124.300003,125.599998,125.349998,22303327,125.349998


In [ ]:
# Download last month data
NoData = []
for i in df['Ticker']:
    try:
        stock = stock_data(i, last_month_start, last_month_end)
        stock.to_excel('Last Month Data/'+i+'.xlsx')
    except:
        NoData.append(i)
        print()
        print(f'No Data for stock {i}')
        print()